In [339]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
pd.set_option('display.max_rows',10)

lista_empresas=pd.read_csv('./lista_empresas_volume_acima_0.3.txt', names=['idt','ticker','name'], encoding = 'utf-8', header=None, delimiter=';')

In [341]:
old = yf.Ticker("PETR4.SA").history(period="1y")
old = old.reset_index()
old

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-02-27,25.718091,26.528033,24.888153,25.298122,100112100,0.0,0
1,2020-02-28,25.158134,25.558104,24.618175,25.338121,89095300,0.0,0
2,2020-03-02,25.498108,26.748015,25.478109,26.528032,71596400,0.0,0
3,2020-03-03,26.568029,27.477961,25.758090,26.048067,86855700,0.0,0
4,2020-03-04,26.908005,26.938004,26.238055,26.888006,42320500,0.0,0
...,...,...,...,...,...,...,...,...
238,2021-02-12,27.809999,28.490000,27.610001,28.440001,47801100,0.0,0
239,2021-02-18,30.379999,30.920000,29.040001,29.270000,105539200,0.0,0
240,2021-02-19,28.030001,28.490000,27.170000,27.330000,215834800,0.0,0
241,2021-02-22,22.799999,23.200001,21.400000,21.670000,490230400,0.0,0


In [342]:

for i in ['Open', 'High', 'Close', 'Low']: 
      old[i]  =  old[i].astype('float64')
        
        
fig = go.Figure(data=[go.Candlestick(x=old['Date'],
                                   open=old['Open'],
high=old['High'],
low=old['Low'],
close=old['Close'])])
fig.show()

In [302]:
for i in range(len(lista_empresas)):
    #print(lista_empresas.iloc[i].ticker)
    #print(yf.Ticker(lista_empresas.iloc[i].ticker).history(period='1d').round())
    if i>1: break

In [378]:
min(yf.Ticker("PETR4.SA").history(period="1d",interval='1m').Low)#.Volume.cumsum()


22.709999084472656

In [300]:
pd.set_option('display.max_rows',10)
m5 = yf.Ticker("PETR4.SA").history(period="60d",interval="5m").reset_index() #.tail(2).head(1).Close.iloc[0]
#excluindo dias >22/feb pela troca de horário
m5 = m5[m5.Datetime.dt.date < pd.to_datetime("2021-02-22").date()]
m5.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
m5['Volume']=m5.Volume/1000000
m5['vol_acumulado'] = m5.groupby(m5.Datetime.dt.date)['Volume'].cumsum()
m5['hour']=m5.Datetime.dt.hour
m5['minute']=m5.Datetime.dt.minute

In [301]:
m5_media_qtd = pd.merge(
    m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
    ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
    ,how='inner', on=['hour','minute']
         )
m5_media_qtd = m5_media_qtd[m5_media_qtd.qtd_vol_acum > m5_media_qtd.qtd_vol_acum.max()*0.2]
m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute']).round(2)
m5_merge.to_csv('./m5.csv')
m5_merge

,Datetime,Open,High,Low,Close,Volume,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum
0,2020-11-23 11:05:00-02:00,24.05,24.12,24.04,24.10,0.00,0.00,11,5,0.27,33.0
1,2020-11-23 11:10:00-02:00,24.10,24.18,24.07,24.18,2.04,2.04,11,10,1.41,56.0
2,2020-11-23 11:15:00-02:00,24.18,24.33,24.17,24.21,2.95,4.99,11,15,2.64,56.0
3,2020-11-23 11:20:00-02:00,24.20,24.30,24.20,24.28,1.66,6.65,11,20,3.72,56.0
4,2020-11-23 11:25:00-02:00,24.28,24.32,24.22,24.29,1.02,7.67,11,25,4.67,57.0
...,...,...,...,...,...,...,...,...,...,...,...
4054,2021-02-19 16:35:00-02:00,27.23,27.32,27.18,27.27,1.24,161.44,16,35,45.24,58.0
4055,2021-02-19 16:40:00-02:00,27.28,27.34,27.26,27.30,0.66,162.10,16,40,45.78,58.0
4056,2021-02-19 16:45:00-02:00,27.31,27.32,27.27,27.27,0.76,162.85,16,45,46.24,58.0
4057,2021-02-19 16:50:00-02:00,27.27,27.30,27.24,27.29,0.79,163.64,16,50,46.90,58.0


In [288]:
acao="LWSA3.SA"
df = yf.Ticker(acao).history(period="30d")
df['ultima_cotacao'] = yf.Ticker(acao).history(period="1m").tail(1).Close.iloc[0]
df['cotacao_dia_anterior'] = yf.Ticker(acao).history(period="5d").tail(2).head(1).Close.iloc[0]
df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df.Volume = df.Volume/1000000
df['media_vol'] = df.Volume.mean()
df['var_vol'] = ((df.Volume-df.media_vol) / df.media_vol)*100
df['pct_longo'] = (df.ultima_cotacao-df.Low)/df.Close*100
df=df.reset_index()
#df.sort_index(ascending=False).round(2)
df = df.sort_values(by='Date',ascending=True).round(2)

In [289]:
df['vol_acumulado'] = df.groupby([df.Date.dt.month])['Volume'].cumsum()

In [115]:
df

,Date,Open,High,Low,Close,Volume,ultima_cotacao,cotacao_dia_anterior,media_vol,var_vol,pct_longo,vol_acumulado
0,2021-01-08,19.72,20.23,19.34,19.76,4.41,31.46,32.86,9.21,-52.10,61.32,4.41
1,2021-01-11,20.34,22.22,20.10,22.04,14.86,31.46,32.86,9.21,61.38,51.55,19.27
2,2021-01-12,22.41,24.85,22.36,24.52,16.84,31.46,32.86,9.21,82.79,37.09,36.11
3,2021-01-13,25.02,25.77,23.25,25.77,14.87,31.46,32.86,9.21,61.39,31.85,50.98
4,2021-01-14,26.06,27.35,25.38,27.25,10.32,31.46,32.86,9.21,12.05,22.32,61.30
5,2021-01-15,27.25,28.10,26.50,27.07,7.67,31.46,32.86,9.21,-16.78,18.32,68.97
6,2021-01-18,27.37,28.34,27.13,27.68,6.18,31.46,32.86,9.21,-32.95,15.65,75.15
7,2021-01-19,27.75,27.94,26.12,26.28,7.23,31.46,32.86,9.21,-21.46,20.31,82.38
8,2021-01-20,26.73,27.49,26.29,27.35,3.85,31.46,32.86,9.21,-58.23,18.91,86.23
9,2021-01-21,27.50,29.45,27.50,29.40,9.16,31.46,32.86,9.21,-0.54,13.47,95.39
